In [ ]:
!pip install transformers datasets

In [ ]:
!pip install tokenizers seqeval

In [ ]:
pip install accelerate -U

In [ ]:
from datasets import load_dataset

dataset_medical_ner = load_dataset('bc2gm_corpus')

Generating train split:   0%|          | 0/12500 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [ ]:
print(len(dataset_medical_ner['train']))
print(len(dataset_medical_ner['validation']))
print(len(dataset_medical_ner['test']))

12500
2500
5000


In [ ]:
from transformers import BertTokenizerFast, AutoTokenizer


tokenizer = AutoTokenizer.from_pretrained('dmis-lab/biobert-base-cased-v1.2')

In [ ]:
def equal_length(examples, split, lbls):

  for i in range(len(examples)):
    tok = tokenizer(examples[split][i]['tokens'], truncation = True, is_split_into_words = True)
    text = examples[split][i]

    if len(text[lbls]) != len(tok['input_ids']):
      return False

    return True

equal_length(dataset_medical_ner, 'train', 'ner_tags')

In [ ]:
def align_labels_and_tokens(examples, label_all_tokens = True):
  tok_text = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
  labels = []

  for i, label in enumerate(examples['ner_tags']):
    id_word = tok_text.word_ids(batch_index = i)
    prev_id = None
    labels_id = []

    for word_idx in id_word:

      if word_idx is None:
        labels_id.append(-100)

      elif word_idx != prev_id:
        labels_id.append(label[word_idx])

      else:
        labels_id.append(label[word_idx] if label_all_tokens else -100)

      prev_id = word_idx
    labels.append(labels_id)
  tok_text['labels'] = labels

  return tok_text

In [ ]:
tokenized_datasets = dataset_medical_ner.map(align_labels_and_tokens, batched=True)

Map:   0%|          | 0/12500 [00:00<?, ? examples/s]

Map:   0%|          | 0/2500 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [ ]:
def get_label_list_and_count(dataset):
  label_list = dataset['train'].features["ner_tags"].feature.names

  return label_list, len(label_list)

In [ ]:
from transformers import DataCollatorForTokenClassification
from transformers import AutoModelForTokenClassification, AutoModel

label_list, nr_labels = get_label_list_and_count(dataset_medical_ner)

model = AutoModelForTokenClassification.from_pretrained("dmis-lab/biobert-base-cased-v1.2", num_labels = nr_labels)

pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at dmis-lab/biobert-base-cased-v1.2 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# model.config.hidden_act = 'relu'
model.config.dropout = 0.3
model.config.hidden_dropout_prob = 0.3

In [ ]:
# from huggingface_hub import notebook_login

# notebook_login()

In [ ]:
from transformers import TrainingArguments, Trainer
args = TrainingArguments(
output_dir = "fnlp_proiect_bc2gm_FINAL",
evaluation_strategy = "epoch",
learning_rate=1e-6,
per_device_train_batch_size=16,
per_device_eval_batch_size=16,
num_train_epochs=10,
weight_decay=0.01,
push_to_hub = True,

)

In [ ]:
data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
import datasets
metric = datasets.load_metric("seqeval")

<ipython-input-17-bfd0a68a2b4e>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = datasets.load_metric("seqeval")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for seqeval contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/seqeval/seqeval.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [ ]:
import numpy as np

def compute_metrics(eval_preds):
    pred_logits, labels = eval_preds
    pred_logits = np.argmax(pred_logits, axis=2)


    predictions = [
    [label_list[eval_preds] for (eval_preds, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(pred_logits, labels)
    ]

    true_labels = [
      [label_list[l] for (eval_preds, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(pred_logits, labels)
    ]

    results = metric.compute(predictions=predictions, references=true_labels)

    return {
   "precision": results["overall_precision"],
   "recall": results["overall_recall"],
   "f1": results["overall_f1"],
  "accuracy": results["overall_accuracy"],
  }




In [ ]:
id2label = {
    0: "O",
    1: "B-GENE",
    2: "I-GENE",
}
label2id = {
    "O": 0,
    "B-GENE": 1,
    "I-GENE": 2,

}

In [ ]:
model.config.id2label = id2label
model.config.label2id = label2id

In [ ]:
trainer = Trainer(
    model,
    args,
   train_dataset=tokenized_datasets["train"],
   eval_dataset=tokenized_datasets["validation"],
   data_collator=data_collator,
   tokenizer=tokenizer,
   compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.414400,0.172376,0.585708,0.646725,0.614706,0.934073
2,0.158000,0.141275,0.671346,0.764569,0.714932,0.947926
3,0.139900,0.126077,0.720507,0.806343,0.761012,0.954133
4,0.122600,0.117514,0.750594,0.814853,0.781405,0.957815
5,0.114400,0.114274,0.762559,0.829809,0.794764,0.959213
6,0.107700,0.111774,0.771018,0.830067,0.799454,0.960110
7,0.104800,0.111434,0.772047,0.837545,0.803463,0.960444
8,0.098000,0.110276,0.779361,0.839350,0.808244,0.961028
9,0.096200,0.109039,0.787022,0.836643,0.811074,0.961696
10,0.097500,0.109327,0.785155,0.840124,0.811710,0.961560


TrainOutput(global_step=7820, training_loss=0.1374206340526376, metrics={'train_runtime': 2481.2929, 'train_samples_per_second': 50.377, 'train_steps_per_second': 3.152, 'total_flos': 5651414093169936.0, 'train_loss': 0.1374206340526376, 'epoch': 10.0})

In [ ]:
trainer.evaluate(tokenized_datasets['test'])

{'eval_loss': 0.11227958649396896,
 'eval_precision': 0.7754426042261565,
 'eval_recall': 0.8354664041348757,
 'eval_f1': 0.8043362360049761,
 'eval_accuracy': 0.96012004653605,
 'eval_runtime': 32.928,
 'eval_samples_per_second': 151.847,
 'eval_steps_per_second': 9.506,
 'epoch': 10.0}